# Analyse the Outlets in the Dataset by Mankoff (2020)

This notebook loads the outlets from the dataset by Mankoff (2020), selects the ones near the 79° North Glacier (79NG), and shows them on a map of Northeast Greenland.
The map also shows the bathymetry and topography of the area, obtained from the RTopo-2.0.4 dataset.

To use this notebook:
1. Download the [dataset by Mankoff (2020)](https://doi.org/10.22008/FK2/XKQVL7) into a folder `data/Mankoff_2020_v5` (the large NetCDF files are not needed here and can be left out).
2. Download the file `RTopo-2.0.4_30sec_Greenland.nc` from the [dataset by Schaffer et al. (2019)](https://doi.org/10.1594/PANGAEA.905295) into a folder `data`.
3. Download the [GitHub repository GEUS-Glaciology-and-Climate/freshwater](https://github.com/GEUS-Glaciology-and-Climate/freshwater) into a folder `freshwater`.

References:
* Mankoff, Kenneth et al. (2020): _Greenland liquid water discharge from 1958 through 2019,_ Earth System Science Data, https://doi.org/10.5194/essd-12-2811-2020 (paper)
* Mankoff, Kenneth (2020): _Streams, Outlets, Basins, and Discharge [k=1.0],_ GEUS Dataverse, V5 (release 2023), https://doi.org/10.22008/FK2/XKQVL7 (dataset)
* Mankoff, Kenneth (6 October 2023): _Freshwater,_ GitHub, https://github.com/GEUS-Glaciology-and-Climate/freshwater (repository)
* Schaffer, Janin et al. (2019): _An update to Greenland and Antarctic ice sheet topography, cavity geometry, and global bathymetry (RTopo-2.0.4),_ PANGAEA, https://doi.org/10.1594/PANGAEA.905295 (dataset)

Notebook by Markus Reinert (IOW, 2023, https://orcid.org/0000-0002-3761-8029).

In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cmocean
from pyproj import Transformer

from freshwater.discharge import discharge as Discharge

## Set up the coordinate transformation

In [ ]:
transform_xy_to_lonlat = Transformer.from_crs("EPSG:3413", "EPSG:4326", always_xy=True).transform

## Load the bathymetry data

In [ ]:
rtopo = xr.open_dataset("data/RTopo-2.0.4_30sec_Greenland.nc")
rtopo = rtopo.set_index(londim="lon", latdim="lat")
rtopo = rtopo.rename({"latdim": "lat", "londim": "lon"})
rtopo = rtopo.sel(lon=slice(-29, -16), lat=slice(78.5, 80.5))
rtopo

## Load the outlet data

In [ ]:
# Choose between 79NG (True) or Zachariae Isstrom (False)
SELECT_79NG = True
if SELECT_79NG:
    position = "-22,79.4"
else:
    position = "-22,79.0"
outlets = Discharge("data/Mankoff_2020_v5/freshwater/", position, quiet=False).outlets()
outlets

## Run some basic checks

In [ ]:
# Check that coast coordinates of ice outlet are equal to coordinates of land outlet
# and that land outlet has no coast coordinates
outlet_ice = outlets[outlets.domain == "ice"]
outlet_land = outlets[outlets.domain == "land"]
if len(outlet_ice) == len(outlet_land) == 1:
    assert outlet_ice.coast_x.iloc[0] == outlet_land.x.iloc[0]
    assert outlet_ice.coast_y.iloc[0] == outlet_land.y.iloc[0]
    assert outlet_ice.coast_lon.iloc[0] == outlet_land.lon.iloc[0]
    assert outlet_ice.coast_lat.iloc[0] == outlet_land.lat.iloc[0]
    assert all(np.isnan(outlet_land.coast_lon))
    assert all(np.isnan(outlet_land.coast_lat))
else:
    print("These checks are only implemented for exactly one ice and one land outlet.")

In [ ]:
# Check that the coordinate transformation works
lon, lat = transform_xy_to_lonlat(outlets.x, outlets.y)
assert np.allclose(lon, outlets.lon) and np.allclose(lat, outlets.lat)

## Show outlets and bathymetry

In [ ]:
plt.figure(dpi=150)
rtopo.bedrock_topography.plot(cmap=cmocean.cm.topo)
for i, outlet in outlets.iterrows():
    try:
        l, = plt.plot(
            *transform_xy_to_lonlat(*outlet.basin.exterior.xy),
            "--" if outlet.domain == "ice" else "-",
            label=outlet.domain,
        )
    except AttributeError as e:
        print(f"AttributeError at oulet #{i}:", e)
    plt.plot(outlet.lon, outlet.lat, "o", color=l.get_color())
plt.legend(title="Domain:", loc="upper left")
plt.title("Outlets around 79NG on RTopo bathymetry")
plt.grid()
filename = f"figures/outlets_{'79NG' if SELECT_79NG else 'ZI'}.png"
print("Saving as", filename)
plt.savefig(filename, dpi=300)